In [0]:
import os
import sys
from google.colab import drive

google_drive_prefix = '/content/drive'
drive.mount(google_drive_prefix) # Mount Google Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
project_directory = '/content/drive/My Drive/Juho Sun'
sys.path.insert(0, project_directory) # add class folder to sys.path

In [0]:
import pandas as pd
accelerometer = pd.read_csv('/content/drive/My Drive/TermProject_Indoor-Outdoor-Detection/Data/Joseph_191122/Accelerometer.csv')
rss = pd.read_csv('/content/drive/My Drive/TermProject_Indoor-Outdoor-Detection/Data/Joseph_191122/Rss.csv')

In [0]:
accelerometer = accelerometer.drop('timeNanos', axis=1)
accelerometer.head()
rss.head()

,bssid,frequency,rss,ssid,timeMillis,timestamp
0,00246c7def92,5240,-87,Welcome_1st_KAIST,1574385251822,1292271413020
1,00246c7def91,5240,-93,eduroam,1574385251822,1292271413012
2,00246c7def90,5240,-88,Welcome_KAIST,1574385251822,1292271413008
3,88366c27e788,2472,-84,CELT_STUDIO_2G,1574385251822,1292271413002
4,d8c7c8df01a2,2472,-92,Welcome_1st_KAIST,1574385251822,1292271412998


In [0]:
import numpy as np

ACC_MAG = accelerometer.assign(
    mag=lambda x: np.sqrt(x['x'] ** 2 + x['y'] ** 2 + x['z'] ** 2)
)

ACC_MAG.head()

,timeMillis,x,y,z,mag
0,1574385249607,0.733200,3.317205,9.394914,9.990287
1,1574385249615,0.690028,3.348856,9.326383,9.933398
2,1574385249625,0.712055,3.424503,9.294100,9.930485
3,1574385249635,0.731084,3.421104,9.062369,9.714163
4,1574385249645,0.709325,3.377222,9.406301,10.019345


In [0]:
def floor_to_label(floor):
    return floor.map({
        -1: 'start',
        -5: 'MOI',
        -4: 'MII',
        -3: 'MIO',
        1: 'AOI',
        2: 'AII',
        3: 'AIO',
        6: 'ABOI',
        7: 'ABII',
        8: 'ABIO'
    })

In [0]:
transition = pd.read_csv('/content/drive/My Drive/TermProject_Indoor-Outdoor-Detection/Data/Joseph_191122/Transition.csv')

LABEL = transition.assign(label = lambda x: floor_to_label(x['floor']))
LABEL = LABEL.drop('floor', axis=1)
LABEL.head()

,timeMillis,label
0,1574385249558,start
1,1574385292404,MOI
2,1574385297110,AII
3,1574385333303,ABII
4,1574385346916,ABIO


In [0]:
gps = pd.read_csv('/content/drive/My Drive/TermProject_Indoor-Outdoor-Detection/Data/Joseph_191122/Real_GPS.csv')

from datetime import datetime

gps = gps.assign(timeMillis = lambda x: x['time'].apply(lambda y: datetime.strptime(y, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp() * 1000))


In [0]:
ACC_LABELED = ACC_MAG.assign(label = 'Undefined')
state = 'OUT'
previous = 0

for row in LABEL.itertuples():

    ACC_LABELED.loc[
        lambda x: (x.timeMillis <= row.timeMillis) & (x.timeMillis >= previous), 'label'
    ] = state

    previous = row.timeMillis

    if (row.label in ('MOI', 'AOI', 'ABOI')):
        state = 'IN'
    if (row.label in ('MIO', 'AIO', 'ABIO')):
        state = 'OUT'

ACC_LABELED.head()

,timeMillis,x,y,z,mag,label
0,1574385249607,0.733200,3.317205,9.394914,9.990287,OUT
1,1574385249615,0.690028,3.348856,9.326383,9.933398,OUT
2,1574385249625,0.712055,3.424503,9.294100,9.930485,OUT
3,1574385249635,0.731084,3.421104,9.062369,9.714163,OUT
4,1574385249645,0.709325,3.377222,9.406301,10.019345,OUT


In [0]:
RSS_SUB = rss.assign(
    timestamps=lambda x: x['timeMillis'] - min(x['timeMillis'])
).sort_values('timestamps')

RSS_SUB = RSS_SUB.loc[
  lambda x: (x['frequency'] >= 5000) & (x['frequency'] <= 6000)
]

RSS_SUB = RSS_SUB.drop('timeMillis', axis=1)

In [0]:
GPS_SUB = gps.assign(
    timestamps=lambda x: x['timeMillis'] - min(x['timeMillis'])
).sort_values('timestamps')

GPS_SUB = GPS_SUB.drop('timeMillis', axis=1)

GPS_SUB = GPS_SUB[['accuracy', 'timestamps']]

In [0]:
RSS_SUB = RSS_SUB.drop(['timestamp', 'ssid', 'frequency', 'bssid'], axis=1)
RSS_SUB.head()

,rss,timestamps
0,-87,0
1,-93,0
2,-88,0
28,-92,4261
72,-92,8542


In [0]:
RSS_MEAN = RSS_SUB.groupby(['timestamps']).mean()
RSS_MEAN.head()

,rss
timestamps,
0,-89.333333
4261,-92.000000
8542,-93.500000
12846,-89.400000
17164,-85.416667


In [0]:
ACC_SUB = ACC_LABELED.assign(
    timestamps=lambda x: x['timeMillis'] - min(x['timeMillis'])
).sort_values('timestamps')

ACC_SUB = ACC_SUB.drop('timeMillis', axis=1)

ACC_SUB

,x,y,z,mag,label,timestamps
0,0.733200,3.317205,9.394914,9.990287,OUT,0
1,0.690028,3.348856,9.326383,9.933398,OUT,8
2,0.712055,3.424503,9.294100,9.930485,OUT,18
3,0.731084,3.421104,9.062369,9.714163,OUT,28
4,0.709325,3.377222,9.406301,10.019345,OUT,38
...,...,...,...,...,...,...
339373,1.438580,3.831297,12.931419,13.563552,Undefined,3393728
339374,0.880310,3.440880,12.495272,12.990243,Undefined,3393738
339375,0.568230,3.340572,12.009000,12.477916,Undefined,3393748
339376,1.060459,3.709777,10.789145,11.458301,Undefined,3393759


In [0]:
import numpy as np
import pandas as pd

#@title Setting
WIN_SIZE_IN_MS = 3000
OVERLAP_RATIO = 0.5
START_TIME, END_TIME = ACC_SUB.loc[:, 'timestamps'].min(), ACC_SUB.loc[:, 'timestamps'].max()

# arange([start, ]stop, [step, ]dtype=None)
# find the end time of each window by considering the overlapping ratio 
WINDOWS = np.arange(START_TIME + WIN_SIZE_IN_MS, END_TIME, WIN_SIZE_IN_MS * (1 - OVERLAP_RATIO))

# new dataframe 
FEATURES_TIME = []
for w in WINDOWS:
  # for a given window, set the start and end time stamps 
  win_start, win_end = w - WIN_SIZE_IN_MS, w

  for var in ['x', 'y', 'z', 'mag']:
    # select the rows that belong to the current window, w
    value = ACC_SUB.loc[
              lambda x: (x['timestamps'] >= win_start) & (x['timestamps'] < win_end), var
            ].values
  
    # extract basic features 
    min_v = np.min(value) # min
    max_v = np.max(value) # max
    mean_v = np.mean(value) # mean
    std_v = np.std(value) # std. dev.
    
    # append each result (w: current window's end-timestamp, extracted feature) as a new row
    FEATURES_TIME.append((w, '{}-{}'.format('Min', var), min_v))
    FEATURES_TIME.append((w, '{}-{}'.format('Max', var), max_v))
    FEATURES_TIME.append((w, '{}-{}'.format('Mean', var), mean_v))
    FEATURES_TIME.append((w, '{}-{}'.format('Std', var), std_v))

# FEATURES_TIME looks like this <timestamp, feature name, value> 
# (4000.0, 'Min-x', -1.848), (4000.0, 'Max-x', -1.659), (4000.0, 'Mean-x', -1.7706786155747838), (4000.0, 'Std-x', 0.023614236975760955)
    
# Create a dataframe w/ column names: timestamps, feature, value  
FEATURES_TIME = pd.DataFrame(FEATURES_TIME, columns=['timestamps', 'feature', 'value'])

# Reshape data to produce a pivot table based on column values
FEATURES_TIME = FEATURES_TIME.pivot(index='timestamps', columns='feature', values='value').reset_index()
FEATURES_TIME.head()

feature,timestamps,Max-mag,Max-x,Max-y,Max-z,Mean-mag,Mean-x,Mean-y,Mean-z,Min-mag,Min-x,Min-y,Min-z,Std-mag,Std-x,Std-y,Std-z
0,3000.0,12.906933,2.017920,3.581410,12.725182,9.845900,0.171266,2.422734,9.454642,7.564434,-1.954923,0.472922,7.141875,1.263868,0.997981,0.710107,1.322742
1,4500.0,12.871655,1.795239,3.889620,12.672749,9.601019,-0.268715,2.513812,9.194480,7.562359,-1.925235,1.032504,6.755642,1.366444,0.798716,0.707250,1.406653
2,6000.0,13.342903,1.823893,3.889620,13.286450,10.004390,-0.111486,2.406736,9.618436,7.470192,-3.015114,0.580766,6.589906,1.509206,0.952475,0.837037,1.561593
3,7500.0,13.903320,2.187113,3.719124,13.716698,9.979076,-0.159785,1.872992,9.709688,6.672065,-3.015114,0.453079,6.362591,1.731509,1.095377,0.809721,1.706726
4,9000.0,14.936322,3.694263,3.719124,14.536320,9.843909,0.295097,1.482339,9.636672,6.672065,-2.090377,0.209502,6.362591,1.984750,1.090044,0.768001,1.978356


In [0]:
import numpy as np
import pandas as pd

#@title Setting
WIN_SIZE_IN_MS = 1000
OVERLAP_RATIO = 0.5
START_TIME, END_TIME = RSS_SUB.loc[:, 'timestamps'].min(), RSS_SUB.loc[:, 'timestamps'].max()

# arange([start, ]stop, [step, ]dtype=None)
# find the end time of each window by considering the overlapping ratio 
WINDOWS = np.arange(START_TIME + WIN_SIZE_IN_MS, END_TIME, WIN_SIZE_IN_MS * (1 - OVERLAP_RATIO))

# new dataframe 
RSS_FEATURES = []
for w in WINDOWS:
  # for a given window, set the start and end time stamps 
  win_start, win_end = w - WIN_SIZE_IN_MS, w

  for var in ['rss']:
    # select the rows that belong to the current window, w
    value = RSS_SUB.loc[
              lambda x: (x['timestamps'] >= win_start) & (x['timestamps'] < win_end), var
            ].values
  
    # extract basic features 
    # min_v = np.min(value) # min
    # max_v = np.max(value) # max
    mean_v = np.mean(value) # mean
    # std_v = np.std(value) # std. dev.
    
    # append each result (w: current window's end-timestamp, extracted feature) as a new row
    # FEATURES_TIME.append((w, '{}-{}'.format('Min', var), min_v))
    # FEATURES_TIME.append((w, '{}-{}'.format('Max', var), max_v))
    RSS_FEATURES.append((w, '{}-{}'.format('Mean', var), mean_v))
    # FEATURES_TIME.append((w, '{}-{}'.format('Std', var), std_v))

# FEATURES_TIME looks like this <timestamp, feature name, value> 
# (4000.0, 'Min-x', -1.848), (4000.0, 'Max-x', -1.659), (4000.0, 'Mean-x', -1.7706786155747838), (4000.0, 'Std-x', 0.023614236975760955)
    
# Create a dataframe w/ column names: timestamps, feature, value  
RSS_FEATURES = pd.DataFrame(RSS_FEATURES, columns=['timestamps', 'feature', 'value'])

# Reshape data to produce a pivot table based on column values
RSS_FEATURES = RSS_FEATURES.pivot(index='timestamps', columns='feature', values='value').reset_index()
RSS_FEATURES.head()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


feature,timestamps,Mean-rss
0,1000.0,-89.333333
1,1500.0,NaN
2,2000.0,NaN
3,2500.0,NaN
4,3000.0,NaN


In [0]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaled = MinMaxScaler().fit_transform(FEATURES_TIME.drop('timestamps', axis=1).to_numpy())
# fit: Compute the minimum and maximum to be used for later scaling.
# transform: Scaling features of X according to feature_range.
# fit_transform: fit & transform at the same time 
# both input/output are numpy arrays, and thus, DataFrame needs to be converted to a NumPy array (by callig to_numpy()) 

# DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)[source] 
FEATURES_TIME_SCALED = pd.DataFrame(
  np.column_stack([FEATURES_TIME.loc[:, 'timestamps'].values, scaled]),
  columns=FEATURES_TIME.columns
)

FEATURES_TIME_SCALED.head()

feature,timestamps,Max-mag,Max-x,Max-y,Max-z,Mean-mag,Mean-x,Mean-y,Mean-z,Min-mag,Min-x,Min-y,Min-z,Std-mag,Std-x,Std-y,Std-z
0,3000.0,0.318616,0.090101,0.355733,0.418705,0.583547,0.100870,0.376053,0.826553,0.580334,0.157541,0.304199,0.714356,0.387772,0.487936,0.532985,0.406156
1,4500.0,0.315158,0.066893,0.406289,0.414167,0.352841,0.026801,0.399178,0.730223,0.579970,0.161584,0.397237,0.665555,0.420418,0.385696,0.530755,0.432852
2,6000.0,0.361352,0.069879,0.406289,0.467282,0.732864,0.053270,0.371991,0.887201,0.563768,0.013186,0.322130,0.644614,0.465854,0.464587,0.632096,0.482146
3,7500.0,0.416288,0.107734,0.378322,0.504519,0.709015,0.045139,0.236471,0.920989,0.423474,0.013186,0.300900,0.615892,0.536605,0.537908,0.610767,0.528320
4,9000.0,0.517549,0.264806,0.378322,0.575456,0.581672,0.121717,0.137282,0.893953,0.423474,0.139098,0.260402,0.615892,0.617203,0.535171,0.578191,0.614740


In [0]:
import plotly.graph_objs as go

fig = go.Figure()

# For feature plotting 
for var in FEATURES_TIME_SCALED.drop(['timestamps'], axis=1).columns:
  fig.add_trace(
      go.Scatter(
          x=FEATURES_TIME_SCALED.loc[:, 'timestamps'],
          y=FEATURES_TIME_SCALED.loc[:, var],
          name=var
      )
  )

for var in RSS_MEAN.columns:
  fig.add_trace(
      go.Scatter(
          x=RSS_MEAN.index,
          y=RSS_MEAN.loc[:, var],
          name=var
      )
  )
  
for var in GPS_SUB.drop(['timestamps'], axis=1).columns:
  fig.add_trace(
      go.Scatter(
          x=GPS_SUB.loc[:, 'timestamps'],
          y=GPS_SUB.loc[:, var],
          name=var
      )
  )

act_start = ACC_SUB.loc[lambda x: x['label'] != x.shift(1)['label'], :]
act_end = ACC_SUB.loc[lambda x: x['label'] != x.shift(-1)['label'], :]



fig.show()